# Deep Learning for text

In [ ]:
import numpy as np

In [ ]:
vocabulary = {}
def one_hot_encode_token(token):
  vector = np.zeros((len(vocabulary, )))
  token_index = vocabulary[token]
  vector[token_index] = 1
  return vector

# **Usando a camada TextVectorization**

Cada etapa que apresentei até agora seria muito fácil de implementar em Python puro. Talvez você pudesse escrever algo assim:

In [ ]:
import string

class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text
                       if char not in string.punctuation)

    def tokenize(self, text):
        text = self.standardize(text)
        return text.split()

    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

vectorizer = Vectorizer()
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
vectorizer.make_vocabulary(dataset)

In [ ]:
test_sentence = "I write, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)


[2, 3, 7, 1, 5, 6]


In [ ]:
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

i write and [UNK] rewrite again


No entanto, usar algo assim não teria muito desempenho. Na prática, você trabalhará com o Keras TextVectorization camada, que é rápida e eficiente e pode ser descartada diretamente em um tf.datapipeline ou um modelo Keras.

Isto é o que TextVectorization camada se parece com:

In [ ]:
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(
    output_mode="int", # Configura a camada para retornar sequências de palavras codificadas como índices inteiros
)

Por padrão, o TextVectorization A camada usará a configuração “converter para minúsculas e remover pontuação” para padronização de texto e “dividir em espaço em branco” para tokenização. Mas o mais importante é que você pode fornecer funções personalizadas para padronização e tokenização, o que significa que a camada é flexível o suficiente para lidar com qualquer caso de uso. Observe que essas funções personalizadas devem operar em tf.stringtensores, não strings Python regulares! Por exemplo, o comportamento da camada padrão é equivalente ao seguinte:

In [ ]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
  lowercase_string = tf.strings.lower(string_tensor) # Converta strings para minúsculas.
  return tf.strings.regex_replace( # Substitua os caracteres de pontuação pela string vazia
      lowercase_string, f"[{re.escape(string.punctuation)}]", "")


def custom_split_fn(string_tensor):
  return tf.strings.split(string_tensor)

text_vectorization = TextVectorization(
    output_mode="int",
    standardize=custom_standardization_fn,
    split=custom_split_fn,
)

Para indexar o vocabulário de um corpus de texto, basta chamar o método `adapt()` da camada com um Datasetobjeto que produz strings, ou apenas com uma lista de strings Python

In [ ]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
text_vectorization.adapt(dataset)

Observe que você pode recuperar o vocabulário computado via get_vocabulary()—isso pode ser útil se você precisar converter texto codificado como sequências inteiras de volta em palavras. As duas primeiras entradas no vocabulário são o token de máscara (índice 0) e o token OOV (índice 1). As entradas na lista de vocabulário são classificadas por frequência, portanto, com um conjunto de dados do mundo real, palavras muito comuns como “o” ou “a” viriam primeiro.

In [ ]:
# Exibindo o vocabulario
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

Para uma demonstração, vamos tentar codificar e depois decodificar uma frase de exemplo:

In [ ]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)


tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [ ]:
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


#1 Preparando os dados de resenhas de filmes do IMDB

Vamos começar baixando o conjunto de dados da página de Andrew Maas em Stanford e descompactando-o:


In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  6744k      0  0:00:12  0:00:12 --:--:-- 5580k


Você fica com um diretório chamado aclImdb, com a seguinte estrutura:

aclImdb/

...train/

......pos/

......neg/

...test/

......pos/

......neg/


Por exemplo, o diretório train/pos/ contém um conjunto de 12.500 arquivos de texto, cada um dos quais contém o corpo do texto de uma crítica de filme com sentimento positivo para ser usado como dados de treinamento. As avaliações de sentimento negativo residem nos diretórios “neg”. No total, são 25 mil arquivos de texto para treinamento e outros 25 mil para teste.

In [ ]:
# excluindo subdiretorio train/unsup
!rm -r aclImdb/train/unsup

Dando uma olhada no conteúdo de alguns dos arquivos de texto

In [ ]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

A seguir, vamos preparar um conjunto de validação separando 20% dos arquivos de texto de treinamento em um novo diretório, aclImdb/val:

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files) # embaralhar a lista de arquivos com uma semente de 1337
    num_val_samples = int(0.2 * len(files)) # pegar 20% dos arquivos de treino para usar na validação
    val_files = files[-num_val_samples:]
    for fname in val_files: # Mova os arquivos para aclImdb/val/neg e aclImdb/val/pos.
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

`os.makedirs` -> criação de diretórios (pastas) em um sistema de arquivos.

`os.listdir`-> listar os arquivos no diretório especificado, no nosso caso, seria no diretório train_dir.

`-num_val_samples` -> indica que estamos pegando os ultimos `num_val_samples` elementos da lista `files`.

`shutil` -> o módulo shutil realiza operações de movimento e cópia de arquivos

`shutil.move()` -> move arquivos ou diretórios de um local para o outro. Isso garante que os mesmos arquivos não fiquem na pasta de treinamento

Lembre-se de como, no capítulo 8, usamos o image_dataset_from_directoryutilitário para criar um lote Datasetde imagens e seus rótulos para uma estrutura de diretórios? Você pode fazer exatamente a mesma coisa para arquivos de texto usando o text_dataset_from_directoryUtilitário. Vamos criar três Datasetobjetos para treinamento, validação e teste:

In [ ]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


Esses conjuntos de dados geram entradas que são TensorFlow tf.string tensores e alvos que são int32 tensores que codificam o valor “0” ou “1”.

In [ ]:
# exibindo as formas e tipos do primeiro lote
for inputs, targets in train_ds:
  print("inputs.shape: ", inputs.shape)
  print("inputs.dtype: ", inputs.dtype)
  print("targets.shape: ", targets.shape)
  print("targets.dtype: ", targets.dtype)
  print("inputs[0]: ", inputs[0])
  print("targets[0]: ", targets[0])
  break

inputs.shape:  (32,)
inputs.dtype:  <dtype: 'string'>
targets.shape:  (32,)
targets.dtype:  <dtype: 'int32'>
inputs[0]:  tf.Tensor(b"Based on a true story of how a man ahead of his time - the great 19th century American poet and humanist Walt Whitman - made a significant contribution to how western medicine treats people with psychological problems.<br /><br />Interested in the treatment of people with psychological problems, he began to associate with psychiatric workers and patients. After seeing the psychological methods of the time (inhumane and ignorantly cruel methods), Walt rejected those methods, and treated patients with compassion and dignity, encouraging other people to do the same> The story of Walt's interactions with psychiatric workers, patients and townsfolk is full of drama, good humor and wisdom. : )", shape=(), dtype=string)
targets[0]:  tf.Tensor(1, shape=(), dtype=int32)


#Processando palavras como um conjunto: A abordagem do saco de palavras

Primeiro, vamos processar nossos conjuntos de dados de texto bruto com um TextVectorization camada para que produzam vetores de palavras binárias com codificação multi-hot. Nossa camada examinará apenas palavras isoladas (ou seja, unigramas ).



In [ ]:
# Pré-processando nosso dataset com uma camada TextVectorization
text_vectorization = TextVectorization(
    max_tokens = 20000, # Em geral 20000 é o tamanho de vocabulário correto para classificação de texto
    output_mode="multi_hot", # codificando os tokens de saída como vetores binários multi-hot
)

text_only_train_ds = train_ds.map(lambda x, y: x) #Prepare um conjunto de dados que produza apenas entradas de texto bruto (sem rótulos).
text_vectorization.adapt(text_only_train_ds) # Use esse conjunto de dados para indexar o vocabulário do conjunto de dados por meio do método adapt().

# Prepare versões processadas de nosso conjunto de dados de treinamento, validação e teste.
binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4
)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4
)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4
)


`tf.keras.layers.TextVectorization`-> Uma camada de pré-processamento que mapeia recursos de texto para sequências inteiras

`text_only_train_ds` -> Cria um novo conjunto de dados, apenas com os dados de texto, descartando as labels (rótulos de classe). Ele é gerado mapeando o conjunto de dados de treinamento (train_ds) e mantendo apenas os dados de entrada

`adapt()` -> utilizado para preparar o TextVectorization para o conjunto de dados

`map(lambda x, y: (text_vectorization(x), y)` -> Esta função aplica uma função lambda que recebe um par de entrada (x,y) onde x são os textos e y são os rótulos e retorna um par onde x é substituido pela versão vetorizada do texto através do TextVectorization, mantendo os rótulos y.

Você pode tentar inspecionar a saída de um desses conjuntos de dados.

In [ ]:
# Inspecionando a saída do nosso dataset unigrama binário
for inputs, targets in binary_1gram_train_ds:
  print("inputs.shape: ", inputs.shape)
  print("inputs.dtype: ", inputs.dtype)
  print("targets.shape: ", targets.shape)
  print("targets.dtype: ", targets.dtype)
  print("inputs[0]: ", inputs[0])
  print("targets[0]: ", targets[0])
  break

inputs.shape:  (32, 20000)
inputs.dtype:  <dtype: 'float32'>
targets.shape:  (32,)
targets.dtype:  <dtype: 'int32'>
inputs[0]:  tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]:  tf.Tensor(0, shape=(), dtype=int32)


A seguir, vamos escrever uma função de construção de modelo reutilizável que usaremos em todos os nossos experimentos nesta seção.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

Finalmente, vamos treinar e testar nosso modelo.

In [ ]:
# treinando e testando o modelo unigrama binário
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only=True)
]
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 10s 11ms/step - loss: 0.3994 - accuracy: 0.8304 - val_loss: 0.2763 - val_accuracy: 0.8960
Epoch 2/10
62

### Bigramas com codificação binária



In [ ]:
# Configurando o TextVectorization para retornar bigramas

text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

Vamos testar o desempenho de nosso modelo quando treinado em pacotes de bigramas codificados em binário.

In [ ]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras",
                                    save_best_only=True)
]
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 6s 9ms/step - loss: 0.3892 - accuracy: 0.8404 - val_loss: 0.2628 - val_accuracy: 0.8992
Epoch 2/10
62

Agora estamos obtendo 90,4% de precisão no teste, uma melhoria acentuada! Acontece que a ordem local é muito importante.



### Bigramas com codificação TF-IDF

Se você estiver classificando um texto, saber quantas vezes uma palavra ocorre em uma amostra é fundamental: qualquer crítica de filme suficientemente longa pode conter a palavra “terrível”, independentemente do sentimento, mas uma crítica que contém muitas ocorrências da palavra “terrível” é provavelmente negativo.

In [ ]:
# Configurando o TextVectorization para retornar contagem de tokens

text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="count",
)

TF-IDF pondera um determinado termo considerando a “frequência do termo”, quantas vezes o termo aparece no documento atual, e dividindo-o por uma medida de “frequência do documento”, que estima a frequência com que o termo aparece no conjunto de dados. Você calcularia da seguinte maneira:

In [ ]:
# pega a visão de como seria o tfidf vectorizer:
import math

def tfidf(term, document, dataset):
    term_freq = document.count(term)
    doc_freq = math.log(sum(doc.count(term) for doc in dataset) + 1)
    return term_freq / doc_freq


TF-IDF é tão comum que está embutido no TextVectorizationcamada. Tudo que você precisa fazer para começar a usá-lo é mudar o output_modeargumento para "tf_idf".

In [ ]:
# Configurando o TextVectorization para retornar saídas ponderadas por TF-IDF
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)

Vamos treinar um novo modelo com este esquema

In [ ]:
# A chamada adapt() aprenderá os pesos TF-IDF além do vocabulário.
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras", save_best_only=True)
]

model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 9s 10ms/step - loss: 0.5279 - accuracy: 0.7197 - val_loss: 0.3159 - val_accuracy: 0.8938
Epoch 2/10
259

KeyboardInterrupt: 

### EXPORTANDO UM MODELO QUE PROCESSA STRINGS BRUTAS

Se quisermos exportar um modelo independente deste pipeline, devemos ter certeza de que ele incorpora seu próprio pré-processamento de texto (caso contrário, você teria que reimplementar no ambiente de produção, o que pode ser desafiador ou pode levar a discrepâncias sutis entre os dados de treinamento e os dados de produção). Felizmente, isso é fácil.

Basta criar um novo modelo que reutilize sua camada `TextVectorization` e adiciona a ela o modelo que voce acabou de treinar:



In [ ]:
inputs = keras.Input(shape=(1,), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

O modelo resultante pode processar lotes de strings brutas:


In [ ]:
import tensorflow as tf
raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.2f} percent positive")

89.65 percent positive


# Processando palavras como uma sequência: A abordagem do modelo de sequência

In [ ]:
# preparando conjuntos de dados de sequência inteira
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

In [ ]:

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)


`output_sequence_length` ->Para manter um tamanho de entrada gerenciável, truncaremos as entradas após as primeiras 600 palavras. Esta é uma escolha razoável, uma vez que o comprimento médio da revisão é de 233 palavras e apenas 5% das revisões têm mais de 600 palavras.

A seguir, vamos fazer um modelo. A maneira mais simples de converter nossas sequências inteiras em sequências vetoriais é codificar one-hot os inteiros (cada dimensão representaria um termo possível no vocabulário). Além desses vetores one-hot, adicionaremos um LSTM bidirecional simples.

In [ ]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirection  (None, 64)                5128448   
 al)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5128513 (19.56 MB)
Trainable params: 5128513 (19.56 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 165s 256ms/step - loss: 0.5571 - accuracy: 0.7157 - val_loss: 0.4212 - val_accuracy: 0.8140
Epoch 2/10
388/625 [=================>............] - ETA: 53s - loss: 0.4119 - accuracy: 0.8398

KeyboardInterrupt: ignored

In [ ]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

In [ ]:
# Modelo que usa uma camada Embedding treinada do zero

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()


callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                            save_best_only=True)
]

model.fit(int_train_ds,
          validation_data=int_val_ds,
          epochs=10,
          callbacks=callbacks)

model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

KeyboardInterrupt: ignored

In [ ]:

embedding_layer = layers.Embedding(input_dim=10, output_dim=256, mask_zero=True)
some_input = [
  [4, 3, 2, 1, 0, 0, 0],
  [5, 4, 3, 2, 1, 0, 0],
  [2, 1, 0, 0, 0, 0, 0]]

mask = embedding_layer.compute_mask(some_input)
mask


<tf.Tensor: shape=(3, 7), dtype=bool, numpy=
array([[ True,  True,  True,  True, False, False, False],
       [ True,  True,  True,  True,  True, False, False],
       [ True,  True, False, False, False, False, False]])>

In [ ]:
# Usando a Embedded Layer com a mascara ativada

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_7 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_4 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

KeyboardInterrupt: ignored

## Usando incorporações de palavras pré-treinadas

GLOVE AND WORD2VEC


Primeiro, vamos baixar os embeddings de palavras GloVe pré-computados no conjunto de dados da Wikipédia em inglês de 2014. É um arquivo zip de 822 MB contendo vetores de incorporação de 100 dimensões para 400.000 palavras (ou tokens sem palavras).

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2024-01-12 12:01:07--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-01-12 12:01:07--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-01-12 12:01:07--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

Vamos analisar o arquivo descompactado (um arquivo .txt) para construir um índice que mapeie palavras (como strings) para sua representação vetorial.

In [ ]:
# Analisando o arquivo de word-embeddings do GloVe
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


## **A arquitetura do Transformer**
## Self-Attention

<img src="https://drek4537l1klr.cloudfront.net/chollet2/HighResolutionFigures/figure_11-6.png">

In [ ]:
def self_attention(input_sequence):
  output = np.zeros(shape=input_sequence.shape)
  # iterando sobre cada token na sequencia de entrada
  for i, pivot_vector in enumerate(input_sequence):
    scores = np.zeros(shape=(len(input_sequence),))
    for j, vector in enumerate(input_sequence):
      # calculando o produto escalar (pontução de atenção entre o token e todos os outros tokens)
      scores[j] = np.dot(pivot_vector, vector.T)
    # dimensionando por um fator de normalização e aplicando softmax
    scores /= np.sqrt(input_sequence.shape[1])
    scores = softmax(scores)
    new_pivot_representation = np.zeros(shape=pivot_vector.shape)
    for j, vector in enumerate(input_sequence):
      # pegue a soma de todos os tokens ponderados pelas pontuações de atenção
      new_pivot_representation += vector * scores[j]
    output[i] = new_pivot_representation
  return output






Claro, na prática você usaria uma implementação vetorizada. Keras tem uma camada integrada para lidar com isso: o MultiHeadAttentioncamada. Veja como você o usaria:

Vamos implementar um codificador Transformer e experimentá-lo na tarefa de classificação de sentimento de crítica de filme.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [ ]:
config = layer.get_config()
new_layer = layer.__class__.from_config(config)

NameError: name 'layer' is not defined